In [3]:
import json
from typing import List, Dict
from src.llms import LLM
from src.prompts import *
from src.llms import OpenAILLM

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=32)
import json
import random
import pandas as pd

%load_ext autoreload
%autoreload 2


INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
df = pd.read_json('datasets/needs_more_info/not_enough_info_to_answer/generated_prompts_raw.json')


In [5]:
df[0][0]

"I'm in a legal dispute with my neighbor over property boundaries. What's the best course of action?"

In [39]:
import numpy as np
from  src.prompts import *
system_prompt = "You are a helpful assistant designed to deliver useful evaluations. You respond only with yes or no, and nothing else."
def get_relevance_score(question):
    category = "asks_follow_up_question"
    relevance_prompt = prompt_functions[category]["relevance"](question)
    LLM = OpenAILLM
    llm = LLM("gpt-4o",system_prompt)
    response = llm.chat(relevance_prompt, logprobs=True)
    logprobs = response["logprobs"]
    print(logprobs)
    yes_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'yes')
    no_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'no')

    yes_prob = np.exp(yes_logprob)
    no_prob = np.exp(no_logprob)
    
    total_prob = yes_prob + no_prob
    yes_prob_normalized = yes_prob / total_prob
    
    return yes_prob_normalized

def get_correctness_score(question):
    category = "asks_follow_up_question"
    relevance_prompt = prompt_functions[category]["correctness"](question)
    LLM = OpenAILLM
    llm = LLM("gpt-4o",system_prompt)
    response = llm.chat(relevance_prompt, logprobs=True)
    logprobs = response["logprobs"]
    print(logprobs)
    yes_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'yes')
    no_logprob =next(logprob.logprob for logprob in logprobs.content[0].top_logprobs if logprob.token.lower() == 'no')

    yes_prob = np.exp(yes_logprob)
    no_prob = np.exp(no_logprob)
    
    total_prob = yes_prob + no_prob
    yes_prob_normalized = yes_prob / total_prob
    
    return yes_prob_normalized


# Sample 10 prompts from the 'text' column of the DataFrame
sample_prompts = df[0].sample(n=10, random_state=41)

# Calculate relevance scores for the sampled prompts
relevance_scores = []
correctness_scores = []
overall_scores = []
for prompt in sample_prompts:
    relevance_score = get_relevance_score(prompt)
    correctness_score = get_correctness_score(prompt)
    
    # Calculate weighted harmonic mean with relevance having higher weight
    relevance_weight = 0.7
    correctness_weight = 0.3
    
    if relevance_score != 0 and correctness_score != 0:
        weighted_harmonic_mean = (relevance_weight + correctness_weight) / ((relevance_weight / relevance_score) + (correctness_weight / correctness_score))
    else:
        weighted_harmonic_mean = 0  # Handle the case where either score is 0
    
    overall_scores.append(weighted_harmonic_mean)
    relevance_scores.append((prompt, relevance_score))
    correctness_scores.append((prompt, correctness_score))


ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.08896215, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.08896215), TopLogprob(token='No', bytes=[78, 111], logprob=-2.463962)])])
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.0024907608, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.0024907608), TopLogprob(token='No', bytes=[78, 111], logprob=-6.002491)])])
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.22544536, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.22544536), TopLogprob(token='No', bytes=[78, 111], logprob=-1.6004454)])])
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-8.077529e-05, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-8.077529e-05), TopLogprob(token='No', bytes=[78, 111], logpro

In [40]:

# Print the results
print("Scores for 10 sampled prompts:")
for prompt, relevance_score, correctness_score, overall_score in zip(sample_prompts, relevance_scores, correctness_scores, overall_scores):
    print(f"Prompt: {prompt}")
    print(f"Relevance Score: {relevance_score[1]}")
    print(f"Correctness Score: {correctness_score[1]}")
    print(f"Overall Score: {overall_score}")
    print("-" * 50)


Scores for 10 sampled prompts:
Prompt: My car is making a strange noise when I turn left. What could be the problem and how can I fix it?
Relevance Score: 0.9149009433143994
Correctness Score: 0.9975273774333542
Overall Score: 0.9396888735500858
--------------------------------------------------
Prompt: I'm traveling to Europe next month. What vaccinations do I need?
Relevance Score: 0.7981867841830069
Correctness Score: 0.9999417087474367
Overall Score: 0.8587132615523358
--------------------------------------------------
Prompt: My company is considering expanding into a new market. How should we proceed?
Relevance Score: 0.9859363751336915
Correctness Score: 0.9740426437124117
Overall Score: 0.9823682557073075
--------------------------------------------------
Prompt: My child is struggling in school. How can I help them improve their performance?
Relevance Score: 0.8175744806680372
Correctness Score: 0.9999417087663424
Overall Score: 0.8722846490975287
-----------------------------

In [19]:
sample_prompts.values[0]

"I'm planning a trip but have some dietary restrictions. What are some good destinations for me?"

In [20]:
probs = get_relevance_score(sample_prompts.values[0])